In [20]:
import karpathy_class as kc
import pandas as pd
import os
import math
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_columns', 70)
merged_folder = '..\\data_files\\merged_files\\'

In [21]:
core_match_data = pd.read_parquet(f'{merged_folder}core_match_data.parquet')

In [40]:
if not os.path.exists('model_files\\'):
    os.mkdir('model_files\\')
if os.path.exists('model_files\\category_info.parquet'):
    ALL_CATEGORIES = pd.read_parquet('model_files\\category_info.parquet')
else:
    # ALL_CATEGORIES = pd.DataFrame(columns=['category_name', 'col_str_value', 'model_col_name', 'col_model_value',])
    ALL_CATEGORIES = pd.DataFrame(columns=['category_name', 'col_str_value', 'col_model_value',])

In [41]:
# def add_to_global_categories(new_categories: pd.Series, category_name: str, model_col_name: str, as_hash: bool, ):
def add_to_global_categories(new_categories: pd.Series, category_name: str, as_hash: bool, ):
    global ALL_CATEGORIES
    existing_definitions = ALL_CATEGORIES[ALL_CATEGORIES['category_name'] == category_name]
    add_cats = None
    if existing_definitions.shape[0] == 0:
        # completely new category
        if as_hash:
            add_numbers = new_categories.apply(hash)
        else:
            add_numbers = range(0, new_categories.shape[0])
        add_cats = pd.DataFrame({
            'category_name': category_name,
            'col_str_value': new_categories,
            # 'model_col_name': model_col_name,
            'col_model_value': add_numbers,
        })
    else:
        if existing_definitions.shape[0] < new_categories.shape[0]:
            new_values = new_categories[~new_categories.isin(existing_definitions['col_str_value'])]
            if as_hash:
                add_numbers = new_values.apply(hash)
            else:
                cur_max_category = existing_definitions['col_model_value'].max()
                add_numbers = range(cur_max_category, new_categories.shape[0])
            add_cats = pd.DataFrame({
                'category_name': category_name,
                'col_str_value': new_values,
                # 'model_col_name': model_col_name,
                'col_model_value': add_numbers,
            })
    if add_cats is not None:
        if add_cats['col_str_value'].isin(ALL_CATEGORIES['col_str_value']).any():
            raise Exception(f'Category {category_name} has values that exist in other categories')
        ALL_CATEGORIES = pd.concat((ALL_CATEGORIES, add_cats), ignore_index=True).sort_values(['category_name', 'col_model_value']).reset_index(drop=True)
    return ALL_CATEGORIES[ALL_CATEGORIES['category_name'] == category_name]


def to_model_values(series: pd.Series, category_name: str | None = None, as_hash: bool = False, player_names: bool = False, ) -> pd.DataFrame:
    if as_hash:
        if player_names:
            col_str_value = series.str.replace('(pen)', '', regex=False).str.strip().drop_duplicates()
        else:
            col_str_value = series.str.strip().drop_duplicates()
        # model_col_name = f'{col_prefix}_hash'
    else:
        col_str_value = series.drop_duplicates()
        # model_col_name = f'{col_prefix}_code'
    if category_name is None:
        category_definitions = add_to_global_categories(col_str_value, str(col_str_value.name), as_hash)
    else:
        category_definitions = add_to_global_categories(col_str_value, category_name, as_hash)
    out_df = pd.DataFrame({'col_str_value': series, })
    out_df = out_df.merge(category_definitions, on='col_str_value')
    return out_df

In [42]:
core_match_data.head()

,away_npxG,away_team,home_goals,home_team,home_xAG,home_team_id,league,utc_datetime,home_ast,local_date,away_team_id,link,away_formation,away_ast,home_manager,stage,year,attendance,away_xAG,second_half_stoppage,away_goals,home_npxG,away_manager,first_half_stoppage,home_captain,away_captain,home_formation,home_xG,away_xG,venue,home_possession_frac,away_possession_frac,home_passing_accuracy,away_passing_accuracy,home_passing_accuracy_frac,away_passing_accuracy_frac,home_shots_on_target,away_shots_on_target,home_shots_on_target_frac,away_shots_on_target_frac,home_saves,away_saves,home_saves_frac,away_saves_frac,og_index
0,NaN,Eintracht Braunschweig,2.0,Düsseldorf,NaN,b1278397,Bundesliga 2,2014-08-01 18:30:00+00:00,2.0,2014-08-01,8107958d,https://fbref.com/en/matches/214b4c53/Dusseldo...,4-1-4-1,1.0,Oliver Reck,Matchweek 1,2014.0,"41,667",NaN,0,2.0,NaN,Torsten Lieberknecht,1,None,None,4-2-3-1,NaN,NaN,"Merkur Spielarena, Düsseldorf",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,Aalen,0.0,RB Leipzig,NaN,acbb6a5b,Bundesliga 2,2014-08-02 11:00:00+00:00,0.0,2014-08-02,eb207015,https://fbref.com/en/matches/27a83d3d/RB-Leipz...,4-3-3,0.0,Alexander Zorniger,Matchweek 1,2014.0,"21,354",NaN,0,0.0,NaN,Stephan Ruthenbeck,0,None,None,4-1-2-1-2,NaN,NaN,"Red Bull Arena, Leipzig",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,NaN,Greuther Fürth,1.0,Bochum,NaN,b42c6323,Bundesliga 2,2014-08-02 13:30:00+00:00,0.0,2014-08-02,12192a4c,https://fbref.com/en/matches/6228d604/Bochum-G...,4-2-3-1,0.0,Peter Neururer,Matchweek 1,2014.0,"18,146",NaN,1,1.0,NaN,Frank Kramer,0,None,None,4-4-2,NaN,NaN,"Vonovia Ruhrstadion, Bochum",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,NaN,Ingolstadt 04,1.0,St. Pauli,NaN,54864664,Bundesliga 2,2014-08-02 13:30:00+00:00,0.0,2014-08-02,12eb2039,https://fbref.com/en/matches/ad976a9d/St-Pauli...,4-3-3,0.0,Roland Vrabec,Matchweek 1,2014.0,"26,664",NaN,0,1.0,NaN,Ralph Hasenhüttl,0,None,None,4-4-2,NaN,NaN,"Millerntor-Stadion, Hamburg",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,NaN,Union Berlin,0.0,Karlsruher,NaN,33ba9d7b,Bundesliga 2,2014-08-03 13:30:00+00:00,0.0,2014-08-03,7a41008f,https://fbref.com/en/matches/1f138e5f/Karlsruh...,3-5-2,0.0,Markus Kauczinski,Matchweek 1,2014.0,"18,489",NaN,0,0.0,NaN,Norbert Düwel,0,None,None,4-1-4-1,NaN,NaN,"Wildparkstadion, Karlsruhe",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


In [43]:
to_model_values(core_match_data['away_team'], category_name='team_name', as_hash=True, ).head()

,col_str_value,category_name,col_model_value
0,Eintracht Braunschweig,team_name,3265767712694642729
1,Aalen,team_name,3129445752576596059
2,Greuther Fürth,team_name,-3515683455896570955
3,Ingolstadt 04,team_name,3455717195590380590
4,Union Berlin,team_name,-7346890077142416946


In [44]:
ALL_CATEGORIES[ALL_CATEGORIES['category_name'] == 'formation']

,category_name,col_str_value,col_model_value


In [45]:
to_model_values(core_match_data['home_team'], category_name='team_name', as_hash=True, )
to_model_values(core_match_data['league'], 'league', as_hash=True, )
to_model_values(core_match_data['away_formation'], 'formation', )

Exception: Category formation has values that exist in other categories

In [46]:
no_formation = (core_match_data['home_team'] == core_match_data['home_formation']) |(core_match_data['away_team'] == core_match_data['away_formation'])
no_formation.sum()

72

In [5]:
shots = raw_shots.copy()
all_players = pd.concat(
    [shots["('Unnamed: 1_level_0', 'Player')"],
     shots["('SCA 1', 'Player')"],
     shots["('SCA 2', 'Player')"]],
    ignore_index=True)
category_lookup = {'num_to_str': {}, 'str_to_num': {}}
category_lookup['num_to_str'], category_lookup['str_to_num'] = player_hash(all_players)
all_events = pd.concat(
    [shots["('SCA 1', 'Event')"],
     shots["('SCA 2', 'Event')"],],
    ignore_index=True)
num_to_str_1, str_to_num_1 = to_model_values(all_events, 'event')
num_to_str_2, str_to_num_2 = to_model_values(shots["('Unnamed: 2_level_0', 'Squad')"], 'squad')
num_to_str_3, str_to_num_3 = to_model_values(shots["('Unnamed: 5_level_0', 'Outcome')"], 'outcome')
num_to_str_4, str_to_num_4 = to_model_values(shots["('Unnamed: 7_level_0', 'Body Part')"], 'body_part')
for e in [num_to_str_1, num_to_str_2, num_to_str_3, num_to_str_4]:
    category_lookup['num_to_str'] |= e
for e in [str_to_num_1, str_to_num_2, str_to_num_3, str_to_num_4]:
    category_lookup['str_to_num'] |= e

shots['shot_taker_hash'] = shots["('Unnamed: 1_level_0', 'Player')"].map(category_lookup['str_to_num']['player_hash'])
shots['sca1_player_hash'] = shots["('SCA 1', 'Player')"].map(category_lookup['str_to_num']['player_hash'])
shots['sca2_player_hash'] = shots["('SCA 2', 'Player')"].map(category_lookup['str_to_num']['player_hash'])

shots['sca1_event_code'] = shots["('SCA 1', 'Event')"].map(category_lookup['str_to_num']['event_code']).fillna(-1)
shots['sca2_event_code'] = shots["('SCA 2', 'Event')"].map(category_lookup['str_to_num']['event_code']).fillna(-1)

shots['squad_code'] = shots["('Unnamed: 2_level_0', 'Squad')"].map(category_lookup['str_to_num']['squad_code']).fillna(-1)
shots['outcome_code'] = shots["('Unnamed: 5_level_0', 'Outcome')"].map(category_lookup['str_to_num']['outcome_code']).fillna(-1)
shots['body_part_code'] = shots["('Unnamed: 7_level_0', 'Body Part')"].map(category_lookup['str_to_num']['body_part_code']).fillna(-1)

shots.rename(columns={
    "('minute', '')": 'minute',
    "('Unnamed: 1_level_0', 'Player')": 'shot_taker',
    "('SCA 1', 'Player')": 'sca1_player',
    "('SCA 2', 'Player')": 'sca2_player',
    "('Unnamed: 6_level_0', 'Distance')": "shot_distance",
}, inplace=True)
keep_cols = ~shots.columns.str.contains('\(|\)')

shots_model = shots[shots.columns[keep_cols]].copy()


shots_model['minute_sin'], shots_model['minute_cos'] = shots_model['minute'] * math.pi / 30, shots_model['minute'] * math.pi / 30
shots_model['minute_sin'], shots_model['minute_cos'] = shots_model['minute_sin'].apply(math.sin), shots_model['minute_cos'].apply(math.cos)

In [6]:
# shots_model
# shots_model.drop(columns=['shot_taker', 'sca1_player','sca2_player','link','minute'], inplace=True)

In [7]:
shots_model.head()

,orig_index,shot_taker,shot_distance,sca1_player,sca2_player,minute,link,shot_taker_hash,sca1_player_hash,sca2_player_hash,sca1_event_code,sca2_event_code,squad_code,outcome_code,body_part_code,minute_sin,minute_cos
0,917,Bernd Nehrig,15.0,None,None,3,https://fbref.com/en/matches/d77e8831/Bochum-S...,-2.369467e+18,-9223363241361209652,-9223363241361209652,-1.0,-1.0,262,3,3,0.309017,9.510565e-01
1,917,Waldemar Sobota,21.0,Bernd Nehrig,None,3,https://fbref.com/en/matches/d77e8831/Bochum-S...,6.139948e+18,-2369466617422978571,-9223363241361209652,4.0,-1.0,262,0,1,0.309017,9.510565e-01
2,917,Johannes Wurtz,14.0,Danilo Soares,Selim Gündüz,12,https://fbref.com/en/matches/d77e8831/Bochum-S...,3.945436e+18,-134023173120860209,-1106607500068338639,3.0,3.0,38,2,1,0.951057,3.090170e-01
3,917,Mats Møller Dæhli,15.0,Bernd Nehrig,Daniel Buballa,14,https://fbref.com/en/matches/d77e8831/Bochum-S...,5.228069e+18,-2369466617422978571,7233307682884112458,3.0,3.0,262,0,3,0.994522,1.045285e-01
4,917,Waldemar Sobota,28.0,Aziz Bouhaddouz,Daniel Buballa,15,https://fbref.com/en/matches/d77e8831/Bochum-S...,6.139948e+18,-2515385730228400528,7233307682884112458,3.0,2.0,262,3,3,1.000000,2.832769e-16
